In [ ]:
pip install langchain langchain-community langchain_openai langchain_text_splitters sentence_transformers pypdfium2

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
   ----------------- ---------------------- 1.3/3.0 MB 7.5 MB/s eta 0:00:01
   -------------------------------------- - 2.9/3.0 MB 7.3 MB/s eta 0:00:01
   ---------------------------------------- 3.0/3.0 MB 6.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: C:\Users\82103\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import os
from pydoc import text
from xml.dom.minidom import Document
import langchain
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tomlkit import documents, item
from langchain_community.vectorstores import Chroma

#API Key 설정 
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

#임베딩 모델 초기화 
embeddings_model = OpenAIEmbeddings(model = 'text-embedding-3-small')

#json 파일 로드 
with open("json_file.json", "r", encoding = "uth-8") as f:
    data = json.load(f)

documents = [
    Document(
        page_content=f"질문: {item['Text']}\n답변: {item['Completion']}",
        metadata={"세부전공": item.get("세부전공", ""), "카테고리": item.get("카테고리", "")}
    )
    for item in data if "Text" in item and "Completion" in item
]


In [ ]:
#json-> text 파일을 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

split_texts = text_splitter.split_documents(documents)

#임베딩
embeddings = embeddings_model.embed_documents([i.page_content for i in split_texts])

#결과 확인
print("임베딩 벡터: ", len(embeddings))
print("임베딩 백터 차원: "len(embeddings[0]))



**ChromaDB저장 및 검색**

In [ ]:
# Chroma 벡터스토어에 저장 (저장 폴더: ./chroma_db)
vectorstore = Chroma.from_documents(
    documents=split_texts,
    embedding=embeddings_model,
    persist_directory="./chroma_db" #위치 변경
)

# Chroma에 저장된 내용 영구화 (재시작 후에도 로드 가능)
vectorstore.persist()

# 저장된 DB를 다시 불러오기기
from langchain_community.vectorstores import Chroma

vectorstore = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embeddings_model
)

질문 기반 유사 문서 검색

In [ ]:
query = "3학년 1학기에 어떤 과목을 들어야 할까?"
results = vectorstore.similarity_search(query, k=3)  # 상위 3개 유사 문서 반환

for i, res in enumerate(results):
    print(f"\n--- 결과 {i+1} ---")
    print(res.page_content)
    print("메타데이터:", res.metadata)